### Goal: Generate slices of fmri scans, then use a classfier to train on newly created slices

In [26]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import nibabel as nib
from pathlib import Path


# SLICE GENERATOR

In [ ]:
devices = tf.config.list_physical_devices()
# print("\nDevices: ", devices)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  details = tf.config.experimental.get_device_details(gpus[0])
  print("GPU details: ", details)

In [28]:
test_img_path = os.path.join('data','fmri','beijing')
savepath = os.path.join('outputs','slice-outputs')
fmri_files = Path("data/fmri/beijing").glob("*.nii.gz")
count = 0

def slice_gen(imagename,x):
    
    slice_data = test_img_data[:,:,20,x]

    plt.imshow(slice_data, cmap='gray')
    plt.axis('off')  # Turn off axis labels
    plt.savefig(savepath + "/"+ imagename, bbox_inches='tight', pad_inches=0)
    plt.close()

for files in fmri_files:
    imgname = 'output' + str(count) + '.png'
    test_img = nib.load(str(files))
    test_img_data = test_img.get_fdata()
    #for x in range (0, 3):
    slice_gen(imgname,count)
    count+=1

# CLASSIFIER

In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "outputs/slice-outputs"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

output0.png
output1.png
output10.png
output100.png
output101.png
output102.png
output103.png
output104.png
output105.png
output106.png
output107.png
output108.png
output109.png
output11.png
output110.png
output111.png
output112.png
output113.png
output114.png
output115.png
output116.png
output117.png
output118.png
output119.png
output12.png
output120.png
output121.png
output122.png
output123.png
output124.png
output125.png
output126.png
output127.png
output128.png
output129.png
output13.png
output130.png
output131.png
output132.png
output133.png
output134.png
output135.png
output136.png
output137.png
output138.png
output139.png
output14.png
output140.png
output141.png
output142.png
output143.png
output144.png
output145.png
output146.png
output147.png
output148.png
output149.png
output15.png
output150.png
output151.png
output152.png
output153.png
output154.png
output155.png
output156.png
output157.png
output158.png
output159.png
output16.png
output160.png
output161.png
output162.png
out

In [23]:
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
import keras
import os
import cv2
from PIL import Image
from keras.models import Sequential, Model
from keras.layers import Input, Flatten, Dense, Dropout, Convolution2D, Conv2D, MaxPooling2D, Lambda, GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Activation, AveragePooling2D, Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers.schedules import ExponentialDecay

%matplotlib inline
keras.backend.set_image_data_format('channels_last')

In [24]:
devices = tf.config.list_physical_devices()
# print("\nDevices: ", devices)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  details = tf.config.experimental.get_device_details(gpus[0])
  print("GPU details: ", details)

GPU details:  {'device_name': 'METAL'}


## Before we can begin opening the images for processing, images need to be grayscaled properly first.

In [26]:
from pathlib import Path

directory = os.path.join('outputs','slice-outputs')
image_list = []
slice_outputs = Path(directory).glob('*.png')

# Iterate through the image files
for slice in slice_outputs:
    
    # converting image data to grayscale
    img = cv2.imread(str(slice))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(str(slice), gray)

    # making images operable
    img = Image.open(slice)

    img_array = np.array(img)
    img_array_flat = img_array.flatten()
    image_list.append(img_array_flat)

# Create a dataframe from the list of flattened image arrays
df = pd.DataFrame(image_list)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,164564,164565,164566,164567,164568,164569,164570,164571,164572,164573
0,3,3,3,3,3,3,3,5,5,5,...,8,8,8,8,10,10,10,10,10,10
1,2,2,2,2,2,2,2,1,1,1,...,5,5,5,5,2,2,2,2,2,2
2,2,2,2,2,2,2,2,2,2,2,...,0,0,0,0,1,1,1,1,1,1
3,2,2,2,2,2,2,2,2,2,2,...,4,4,4,4,3,3,3,3,3,3
4,3,3,3,3,3,3,3,5,5,5,...,0,0,0,0,2,2,2,2,2,2


## Pixel size of new images are required to perform cnn

In [41]:
img = "outputs/slice-outputs/output0.png"
img = Image.open(str(slice))

width, height = img.size
print(f'Image dimensions: {width}x{height}')

Image dimensions: 446x369


In [43]:
def load_data_kfold(k):
    # Assuming the DataFrame df already contains flattened image arrays
    X_train = df.values  # Get the image data from the DataFrame as numpy array
    
    # If your images are grayscale or RGB, you may need to reshape them back
    # For example, if your images are 75x75:
    # X_train = np.array([img.reshape(75, 75, 3) for img in X_train])  # for RGB images
    # Or if grayscale:
    # X_train = np.array([img.reshape(75, 75) for img in X_train])
    
    # y_train is your target variable (this needs to be defined if not in the DataFrame)
    # For example, if the labels (is_iceberg) are in another list or DataFrame column:
    # y_train = your_labels  # This should match the size of your data
    
    # Placeholder for example labels (binary classification 0 or 1)
    y_train = np.random.randint(2, size=len(X_train))  # Replace with actual labels
    
    # Create stratified K-Folds
    folds = list(StratifiedKFold(n_splits=k, shuffle=True, random_state=1).split(X_train, y_train))
    
    return folds, X_train, y_train

k = 7
folds, X_train, y_train = load_data_kfold(k)

In [50]:
def get_model():
    
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(369, 446, 1)))  # Update input_shape
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.9)
    
    opt_adam = keras.optimizers.Adam(learning_rate=lr_schedule, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile(opt_adam, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [51]:
model = get_model()
model.summary()

/opt/anaconda3/envs/slice-demo-1/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_20 (Conv2D)              │ (None, 367, 444, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 183, 222, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 181, 220, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 90, 110, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 633600)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │    81,100,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 81,119,873 (309.45 MB)

 Trainable params: 81,119,873 (309.45 MB)

 Non-trainable params: 0 (0.00 B)

In [52]:
batch_size=64

gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1,
                         zoom_range = 0.1,
                         rotation_range = 10
                        )

In [53]:
def get_callbacks(name_weights, patience_lr):
    mcp_save = ModelCheckpoint(name_weights, save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=patience_lr, verbose=1, epsilon=1e-4, mode='min')
    return [mcp_save, reduce_lr_loss]

In [54]:
# First, print the shape of the data to check the size of the flattened images
print(X_train.shape)

# Assuming the flattened images have a different size than 75x75x3, adjust the reshape accordingly
# For example, if the images are grayscale and the size is 75x75:
# Replace 75x75x3 with 75x75x1 if grayscale, or calculate the correct dimensions if otherwise
for j, (train_idx, val_idx) in enumerate(folds):
    print('\nFold ', j)

    # Check the size of the fold
    print(f"Size of X_train_cv before reshaping: {X_train[train_idx].size}")
    print(f"Shape of X_train_cv before reshaping: {X_train[train_idx].shape}")
    print(f"Expected size after reshaping: {468*468*3}")
    
    # Update the dimensions based on the actual shape determined from the output
    height, width, channels = 369, 446, 1  # Adjust based on actual dimensions from the output

    X_train_cv = X_train[train_idx].reshape(-1, height, width, channels)
    y_train_cv = y_train[train_idx]
    X_valid_cv = X_train[val_idx].reshape(-1, height, width, channels)
    y_valid_cv = y_train[val_idx]
    
    name_weights = "final_model_fold" + str(j) + "_weights.keras"
    callbacks = get_callbacks(name_weights=name_weights, patience_lr=10)
    
    generator = gen.flow(X_train_cv, y_train_cv, batch_size=batch_size)
    
    model = get_model()
    
    model.fit(
        generator,
        steps_per_epoch=len(X_train_cv) // batch_size,
        epochs=15,
        shuffle=True,
        verbose=1,
        validation_data=(X_valid_cv, y_valid_cv),
        callbacks=callbacks
    )
    
    print(model.evaluate(X_valid_cv, y_valid_cv))

(194, 164574)

Fold  0
Size of X_train_cv before reshaping: 27319284
Shape of X_train_cv before reshaping: (166, 164574)
Expected size after reshaping: 657072
Epoch 1/15


2024-09-23 15:01:26.557510: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
/opt/anaconda3/envs/slice-demo-1/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.4844 - loss: 40459.7617 - val_accuracy: 0.5714 - val_loss: 553.2520 - learning_rate: 0.0100
Epoch 2/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 131ms/step - accuracy: 0.6053 - loss: 527.9445 - val_accuracy: 0.4286 - val_loss: 8396.2686 - learning_rate: 0.0100
Epoch 3/15


2024-09-23 15:01:32.868308: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-09-23 15:01:32.868327: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 14719937341748615644
2024-09-23 15:01:32.868333: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 7907445620732095406
2024-09-23 15:01:32.868338: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 12704717811458323390
2024-09-23 15:01:32.868343: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 9838118439106987368
2024-09-23 15:01:32.868348: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 3547068773110750812
2024-09-23 15:01:32.868354: W tensorflow/core/framework/local_rendezv

2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 782ms/step - accuracy: 0.4083 - loss: 6695.7217 - val_accuracy: 0.4286 - val_loss: 1400.7307 - learning_rate: 0.0100
Epoch 4/15
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 585ms/step - accuracy: 0.5000 - loss: 1184.0525

...

2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 742ms/step - accuracy: 0.5260 - loss: 0.6979 - val_accuracy: 0.5185 - val_loss: 0.6928 - learning_rate: 0.0100
Epoch 12/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 105ms/step - accuracy: 0.6154 - loss: 0.6832 - val_accuracy: 0.5185 - val_loss: 0.6923 - learning_rate: 0.0100
Epoch 13/15


2024-09-23 15:04:33.896768: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-09-23 15:04:33.896791: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_25]]
2024-09-23 15:04:33.896829: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 16196377949795922081
2024-09-23 15:04:33.896835: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 6193299883451382605
2024-09-23 15:04:33.896838: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 7444028769814597029
2024-09-23 15:04:33.896842: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 4210606093527688421
2024-09-2

2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.5826 - loss: 0.6866 - val_accuracy: 0.5185 - val_loss: 0.6912 - learning_rate: 0.0100
Epoch 14/15
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 654ms/step - accuracy: 0.5781 - loss: 0.6855

2024-09-23 15:04:37.173869: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-09-23 15:04:37.173884: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 3547068773110750812
2024-09-23 15:04:37.173889: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_22]]
2024-09-23 15:04:37.173894: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 16196377949795922081
2024-09-23 15:04:37.173898: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 7928290333029697835
2024-09-23 15:04:37.173903: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 14639214881026905985
2024-09-

2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 990ms/step - accuracy: 0.5781 - loss: 0.6855 - val_accuracy: 0.5185 - val_loss: 0.6907 - learning_rate: 0.0100
Epoch 15/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.5505 - loss: 0.6889 - val_accuracy: 0.5556 - val_loss: 0.6882 - learning_rate: 0.0100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 0.5556 - loss: 0.6882
[0.6881550550460815, 0.5555555820465088]
